In [ ]:
import tnn
import torch
import torch.nn as nn
import logging

from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

loggers = [logging.getLogger(name) for name in logging.root.manager.loggerDict]
for logger in loggers:
    if "transformers" in logger.name.lower():
        logger.setLevel(logging.ERROR)

In [ ]:
dataset = load_dataset("stanfordnlp/imdb")
test = dataset["test"]

In [ ]:
num_labels = len(train.unique("label"))
name = "distilbert/distilbert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(name)

In [ ]:
def pre_process(examples):
    examples = tokenizer(
        examples["text"],
        max_length=512,
        truncation=True,
        padding="max_length",
    )
    return examples


train_dataset = train.map(
    pre_process,
    batched=True,
    num_proc=2,
)
test_dataset = test.map(
    pre_process,
    batched=True,
    num_proc=2,
)

In [ ]:
def collate_fn(batch):
    inputs = {"input_ids": [], "attention_mask": []}
    labels = []
    for example in batch:
        inputs["input_ids"].append(example["input_ids"])
        inputs["attention_mask"].append(example["attention_mask"])
        labels.append(example["label"])

    inputs["input_ids"] = torch.tensor(inputs["input_ids"]).long()
    inputs["attention_mask"] = torch.tensor(inputs["attention_mask"]).long()
    labels = torch.tensor(labels).long()
    return inputs, labels

In [ ]:
testloader = data.DataLoader(
    test_dataset,
    batch_size=32,
    collate_fn=collate_fn,
    drop_last=False,
    shuffle=False,
    pin_memory=True,
    num_workers=2,
)


def to(inputs, labels, device, non_blocking):
    inputs = dict(
        map(
            lambda item: (item[0], item[1].to(device, non_blocking=non_blocking)),
            inputs.items(),
        )
    )
    labels = labels.to(device, non_blocking=True)
    return inputs, labels

In [ ]:
class DistilBert(tnn.Model):

    def __init__(self, name, num_labels):
        super().__init__()
        self.distil_roberta = AutoModelForSequenceClassification.from_pretrained(
            name, num_labels=num_labels
        )

    def forward(self, **inputs):
        return self.distil_roberta(**inputs)

In [ ]:
model = DistilBert(name=name, num_labels=num_labels)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
landscape = tnn.Landscape.from_file(
    trainer_path="../training/imdb-bert-cls-base.h5",
    model=model,
    loss_fn=loss_fn,
    eval_dataloader=testloader,
    device=device,
    path="../images/bert-cls-base-landscape.h5",
    verbose=5,
)

In [ ]:
meshgrid_dict = landscape.create_meshgrid(
    start=-1.0,
    stop=1.0,
    step=10.0,
    use_logspcae=False,
    mode="pca",
)